In [ ]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)

In [ ]:
import os

project_path = 'drive/MyDrive/Сертификат/news-dataset/'

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words('english'))

In [ ]:
from tqdm import tqdm
import pandas as pd
import re
all_lemmas = []
for train_idx in ['1', '2', '3', '4']:
    c = pd.read_csv(f"drive/MyDrive/Сертификат/news-dataset/assets/annotated-corpus/train/{train_idx}.tsv",delimiter='\t')
    sentence_lemmas=[]
    prev_doc_id = c.values[0][0]
    for el in  tqdm(c.values):
        lemma = el[3]
        if el[1]=="\n":
            continue
        if el[0]!=prev_doc_id:
            all_lemmas.append((prev_doc_id, sentence_lemmas))
            sentence_lemmas=[]
            prev_doc_id=el[0]
        else:
            if type(lemma) == str:
                lemma_filtered = re.sub(r'[^\w\s]','', lemma)
                if len(lemma_filtered)==0 or lemma_filtered in stops:
                    continue
                sentence_lemmas.append(lemma_filtered.lower())


In [ ]:
N = len(all_lemmas)

In [ ]:
all_words = [el for sentence in all_lemmas for el in sentence[1]]

In [ ]:
len(all_words)

In [ ]:
from collections import Counter
word_cnt = Counter(all_words)

In [ ]:
len(word_cnt)

In [ ]:
word_cnt = Counter({k: c for k, c in word_cnt.most_common(8192)})

In [ ]:
word_cnt

In [ ]:
import json
with open("tokens_freq.json", "w") as file:
    json.dump(word_cnt, file)

In [ ]:
def create_term_document_matrix(documents, token_freq):
    matrix = []
    token_freq = list(token_freq.keys())
    for doc in tqdm(documents[:60000]):
        tokens = doc[1]
        tokens_cnt = Counter(tokens)
        row = [tokens_cnt[token.lower()] for token in token_freq]
        matrix.append(row)

    return matrix

In [ ]:
len(word_cnt)

In [ ]:
len(all_lemmas)

In [ ]:
term_doc_matrix = create_term_document_matrix(all_lemmas,word_cnt)

In [ ]:
import json
with open("term_document_matrix.json", "w") as file:
    json.dump(term_doc_matrix, file)

In [ ]:
import gc
gc.collect()

In [ ]:
# Перезагрузочка
import numpy as np

In [ ]:
import json
with open("drive/MyDrive/Сертификат/news-dataset/assets/tokens_freq.json", "r") as file:
    word_cnt = json.load(file)

In [ ]:
word_cnt

In [ ]:
#import json
#with open("term_document_matrix.json", "r") as file:
#    term_doc_matrix = json.load(file)

In [ ]:
term_doc_matrix = np.array(term_doc_matrix)

In [ ]:
term_doc_matrix.shape

In [ ]:
from math import log
from tqdm import tqdm
IDF = {}
terms = list(word_cnt.keys())
for i in tqdm(range(len(terms))):
    IDF[i] = log( (1+N) / (1+np.count_nonzero(term_doc_matrix[:,i]) ))

In [ ]:
term_doc_matrix[0].shape

In [ ]:
def compute_tf_idf(term_document_matrix, terms):
    tf_idf_matrix = []
    for document in tqdm(term_document_matrix):
        tf_idf_row=[]
        for i in range(len(terms)):
            tf = document[i]
            tf_idf_row.append(tf * IDF[i])
        tf_idf_matrix.append(tf_idf_row)

    return tf_idf_matrix

In [ ]:
tf_idf_matrix = compute_tf_idf(term_doc_matrix[:10000], terms)

In [ ]:
tf_idf_matrix_np = np.array(tf_idf_matrix)

In [ ]:
tf_idf_matrix_np.shape

In [ ]:
vec = tf_idf_matrix[0]
print(vec)

In [ ]:
# 3

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=[el[1] for el in all_lemmas[:]], vector_size=256, window=5, min_count=1, workers=14)
model.save("word2vec.model")

In [ ]:
model = Word2Vec.load("word2vec.model")

In [ ]:
model.train([el[1] for el in all_lemmas[:]], total_examples=len(all_lemmas[:]), epochs=100)

In [ ]:
from scipy.spatial import distance
1 - distance.cosine(model.wv["winter"], model.wv["summer"])

In [ ]:
np.dot(model.wv["winter"], model.wv["summer"])/(np.linalg.norm(model.wv["winter"])*np.linalg.norm(model.wv["summer"]))

In [ ]:
import math
def cosine_similarity(v1,v2):
    dot_product, norm1, norm2 = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]
        y = v2[i]
        dot_product+=x*y
        norm1 += x*x
        norm2 += y*y
    norm1 = math.sqrt(norm1)
    norm2 = math.sqrt(norm2)
    return dot_product/(norm1*norm2)

In [ ]:
cosine_similarity(model.wv["winter"], model.wv["summer"])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.decomposition import PCA
pca1 = PCA(n_components=512)
tf_idf_matrix_np_transformed = pca1.fit_transform(tf_idf_matrix_np.T)

In [ ]:
from sklearn.decomposition import PCA
words = list(model.wv.key_to_index)
X = [model.wv[word] for i, word in enumerate(words)]
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
for word in ['winter',"snow","summer","hot","cold","bomb","president","cool","heat","sunday","december","july"]:
    vec = model.wv[word]
    x,y = pca.transform(vec.reshape(1, -1))[0][0], pca.transform(vec.reshape(1, -1))[0][1]
    plt.plot(x,y, 'o', color='red')
    plt.annotate(word, (x, y))

In [ ]:
tf_idf_matrix_np = np.array(tf_idf_matrix)

In [ ]:
tf_idf_matrix_np #tf_idf_matrix_np_transformed

In [ ]:
cosine_similarity(tf_idf_matrix_np_transformed[list(word_cnt.keys()).index("election"),:], tf_idf_matrix_np_transformed[list(word_cnt.keys()).index("president"),:])


In [ ]:
cosine_similarity(tf_idf_matrix_np[:,list(word_cnt.keys()).index("election")], tf_idf_matrix_np[:,list(word_cnt.keys()).index("president")])


In [ ]:
cosine_similarity(model.wv["election"], model.wv["president"])


In [ ]:

import numpy as np
def get_sentence_emb(sent):
    sum_vector = np.zeros(256)
    for token in sent:
        try:
            emb = model.wv[token]
        except:
            continue
        sum_vector += emb
    return sum_vector/len(sent)

In [ ]:
get_sentence_emb(all_lemmas[6][1])

In [ ]:
def embedd_csv_text(lemmas,test_idx):
    for el in lemmas:
        docid=el[0]
        text=el[1]
        with open(f"drive/MyDrive/Сертификат/news-dataset/assets/annotated-corpus/train/{test_idx}_emb.tsv", "a", encoding="utf-8") as file:
            embed = get_sentence_emb(text)
            file.write(f"{docid}\t")
            for emb in embed:
                file.write(f"{emb}\t")
            file.write("\n")

In [ ]:
from tqdm import tqdm
import pandas as pd
import re
for test_idx in ['1', '2', '3', '4']:
    test_all_lemmas = []
    c = pd.read_csv(f"drive/MyDrive/Сертификат/news-dataset/assets/annotated-corpus/train/{train_idx}.tsv",delimiter='\t')
    sentence_lemmas=[]
    prev_doc_id = c.values[0][0]
    for el in  tqdm(c.values):
        lemma = el[3]
        if el[1]=="\n":
            continue
        if el[0]!=prev_doc_id:
            test_all_lemmas.append((prev_doc_id, sentence_lemmas))
            sentence_lemmas=[]
            prev_doc_id=el[0]
        else:
            if type(lemma) == str:
                lemma_filtered = re.sub(r'[^\w\s]','', lemma)
                if len(lemma_filtered)==0 or lemma_filtered in stops:
                    continue
                sentence_lemmas.append(lemma_filtered.lower())
    embedd_csv_text(test_all_lemmas,test_idx)


In [ ]:
len(test_all_lemmas)
